## MODIS LAI 500m HDF to GeoTIFF Mosaic

Based on MOD15A2H files

In [ ]:
import os
from pathlib import Path
from json import dumps

In [ ]:
src_data = Path('<path-to-modis-hdfs>')
tile_dir = Path('<path-to-output-folder') / 'tiles'
mos_dir = tile_dir.parent / 'mosaics'
tile_dir.mkdir(parents=True, exist_ok=true)
mos_dir.mkdir(exist_ok=True)

In [ ]:
hdfs = sorted(src_data.glob('*.hdf'))
print('n_datasets: %d' % len(hdfs))
print('example: %s' % hdfs[0])

In [ ]:
unique_tsteps = list(set([x.stem.split('.')[1] for x in hdfs]))
print('unique tsteps: %d' % len(unique_tsteps))
print('example: %s' % unique_tsteps[0])

### convert all hdfs to tifs

In [ ]:
tifs = []
for hdf5 in hdfs:
    sd = f'HDF4_EOS:EOS_GRID:"{str(hdf5)}":MOD_Grid_MOD15A2H:Lai_500m'
    tif = out_data / (hdf5.stem + '.tif')
    if not tif.exists():
        tif_str = str(tif)
        stdout = !gdalwarp -t_srs "+proj=latlong +ellps=sphere" $sd $tif_str
    tifs.append(str(tif))
    print('.', end='', flush=True)

### create mosaics based on timestep

In [ ]:
mosaics = {
    tstep: [t for t in tifs if t.stem.split('.')[1] == tstep]
    for tstep in unique_tsteps
}

In [ ]:
print(dumps({k:len(v) for k,v in mosaics.items()}, indent=2))

In [ ]:
for key in mosaics:
    vrtname = out_data / f'MOD15A2H.{key}.mosaic.vrt'
    mosname = mos_dir / (vrtname.stem + '.tif')
    vrtname_str = str(vrtname)
    mosname_str = str(mosname)
    if not vrtname.exists():
        inputs = ' '.join([str(f) for f in mosaics[key]])
        stdout_vrt = !gdalbuildvrt $vrtname_str $inputs
    if not mosname.exists():
        stdout_mos = !gdal_translate -of GTiff -co "TILED=YES" $vrtname_str $mosname_str
    print(mosname.name)